In [ ]:
#webull option data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
sns.set_theme()

In [ ]:
today=datetime.today().strftime("%m/%d/%y")
today

In [ ]:
f='aug31.csv'
file="/home/thakur/all_stocks/{}".format(f)
df=pd.read_csv(file)[::-1].reset_index(drop=True)

In [ ]:
def get_label(x,y):
    s=x.split();s1=y.split()
    
    return s[0]+"-"+s[4]+"-"+s[-1].split('.')[0]+"-"+s1[1]

In [ ]:
df

In [ ]:
#some cleaning 
def clean_name(x):
    '''
    This function returns the ticker-price-call/put from Name of the dataframe df
    '''
    parts=x.split()
    sel=f'{parts[0]}-{parts[-1][1:-3]}-{parts[-2]}'
    return sel

In [ ]:
def clean_time(x):
    '''
    cleans the Placed/Filled time to get only 00:00:00
    '''
    return x.split()[1]

In [ ]:
df_copy=df.copy()
df_copy['Name']=df_copy['Name'].apply(lambda x:clean_name(x))
df_copy['Placed Time']=df_copy['Placed Time'].apply(lambda x:clean_time(x))
df_copy['Filled Time']=df_copy['Filled Time'].apply(lambda x:clean_time(x))

In [ ]:
df_copy=df_copy.drop(columns=['Status','Filled','Price','Time-in-Force'])
df_copy['Total Price']=df_copy['Total Qty']*df_copy['Avg Price']
df_copy

In [ ]:
#unique transactions
t_list=list(df_copy.Symbol.unique())
t_list

In [ ]:
df_copy.loc[df_copy.Symbol==t_list[0]]

In [ ]:
f_str=22*'=='
main_list=[]
grouped=df_copy.groupby('Symbol')
for t in t_list:
    print(f'\n{f_str} {t} {f_str}\n')
    group=grouped.get_group(t).reset_index(drop=True)
    name=group['Name'].values[0]
    buy=group[group.Side=='Buy']['Total Price'].sum()
    sell=group[group.Side=='Sell']['Total Price'].sum()
    pl=sell-buy
    print(f'{group.to_string()}\n')
    # print(f'buy : {buy*100}')
    # print(f'sell: {sell*100}')
    # print(f'pl  : {pl*100:0.2f}')
    
    f_print=f'''
    SUMMARY:
    {name}:   
   
            buy : {buy*100:0.2f}
            sell: {sell*100:0.2f}
            pl  : {pl*100:0.2f}
    '''
    print(f_print)
    main_list.append([name,t,buy*100,sell*100,pl*100])

In [ ]:
columns=['Name','Symbol','CP','SP','PL']
sum_df=pd.DataFrame(main_list,columns=columns)
#sum_df['CUMSUM']=sum_df['PL'].cumsum()
sum_df

In [ ]:
sum_df.insert(1,'Ticker',sum_df['Name'].apply(lambda x:x.split('-')[0]))
sum_df.insert(2,'Kind',sum_df['Name'].apply(lambda x:x.split('-')[2]))

In [ ]:
sum_df

In [ ]:
cp_list=[]
for i in sum_df.Ticker.unique():
    temp_df=sum_df[sum_df.Ticker==i]
    #print(temp_df)
    count=temp_df.shape[0]
    total_cp=temp_df.CP.sum().round(2)
    total_pl=temp_df.PL.sum().round(2)
    cp_list.append([i,count,total_cp,total_pl])

In [ ]:
tra_sum=pd.DataFrame(cp_list,columns=['TICKER','COUNT','CP','PL'])
tra_sum['CP%']=round(tra_sum['CP']*100/tra_sum.CP.sum(),2)
tra_sum

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.pie(tra_sum.COUNT, labels=tra_sum.TICKER,autopct='%1.1f%%')
plt.title('COUNT')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
tra_sum=tra_sum.set_index('TICKER')
ax=tra_sum['PL'].plot(kind='bar',color=(tra_sum.PL>0).map({True:'g',False:'r'}))
ax.bar_label(ax.containers[0], label_type='edge',color='blue')
plt.show()

In [ ]:
tra_sum

In [ ]:
ax=tra_sum.plot(kind='bar',subplots=True,figsize=(15,10),sharex=False,layout=(2,2))
plt.tight_layout()
#for c in range(len(ax)):ax[c].bar_label(ax[c].containers[0], label_type='edge',color='blue')
ax[0,1].bar_label(ax[0,1].containers[0],label_type='edge',color='blue')
plt.show()


In [ ]:
for i in range(2):
    for j in range(2):
        print(f'{i}-{j}')

In [ ]:
len(ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.pie(tra_sum['CP%'], labels=tra_sum.TICKER,autopct='%1.1f%%')
plt.title('INVESTMENT')
plt.show()

In [ ]:
cum_sum_df=sum_df.copy()
cum_sum_df['CUMSUM']=cum_sum_df.PL.cumsum()
cum_sum_df

In [ ]:
#ticker transaction
tra=cum_sum_df.Ticker.value_counts().to_frame()#.reset_index()
tra.columns=['COUNTS']
tra

In [ ]:
tra.plot(kind='bar',ylabel='COUNTS')

In [ ]:
#ticker transaction
tra=cum_sum_df.Kind.value_counts().to_frame()#.reset_index()
tra.columns=['COUNTS']
tra

In [ ]:
tra.plot(kind='bar',ylabel='COUNTS')

In [ ]:
plt.rcParams['font.size'] = 15
net=round(cum_sum_df.PL.sum(),2)
ax=cum_sum_df.plot(kind='bar',x='Name',y='PL',width=1.,figsize=(16,10),color=(cum_sum_df.PL>0).map({True:'g',False:'r'}))
plt.title(f'Net PL: {net}',color='r' if net<0 else 'b',fontsize=15)
ax.bar_label(ax.containers[0], label_type='edge',color='blue')
ax1=ax.plot(cum_sum_df['CUMSUM'],marker='o')
plt.tight_layout()
xval=list(ax1[0].get_data()[0])
yval=list(ax1[0].get_data()[1])
yval=[round(i,2) for i in yval]
for (x,y) in zip(xval,yval):
    c='r' if y<0 else 'g'
    #print(c)
    plt.text(x,y,str(y),color=c,fontfamily='serif',rotation=30)
plt.show()

In [ ]:
tick_grouped=sum_df.groupby('Ticker')
for tick in list(sum_df.Ticker.unique()):
    group=tick_grouped.get_group(tick).reset_index(drop=True)
    #group.drop(columns=['CUMSUM'],inplace=True)
    #print(group)
    group['CUMSUM']=group['PL'].cumsum()
    #print(group)
    # get_new_plot(group)
    net=round(group.PL.sum(),2)
    ax=group.plot(kind='bar',x='Name',y='PL',width=1.0,figsize=(12,8),color=(group.PL>0).map({True:'g',False:'r'}))
    plt.title(f'{tick} [Net PL: {net}]',color='r' if net<0 else 'g',fontsize=15)
    ax.bar_label(ax.containers[0], label_type='edge',color='blue')
    ax1=ax.plot(group['CUMSUM'],marker='o')
    #ax1.bar_label(ax1.containers[1], label_type='edge')
    xval=list(ax1[0].get_data()[0])
    yval=list(ax1[0].get_data()[1])
    yval=[round(i,2) for i in yval]
    for (x,y) in zip(xval,yval):
        c='r' if y<0 else 'g'
        #print(c)
        plt.text(x,y,str(y),color=c,fontfamily='serif',rotation=0)
    plt.tight_layout()
    plt.show()


In [ ]:
tick_grouped=sum_df.groupby('Kind')
for tick in list(sum_df.Kind.unique()):
    group=tick_grouped.get_group(tick).reset_index(drop=True)
    #group.drop(columns=['CUMSUM'],inplace=True)
    #print(group)
    group['CUMSUM']=group['PL'].cumsum()
    #print(group)
    # get_new_plot(group)
    net=round(group.PL.sum(),2)
    ax=group.plot(kind='bar',x='Name',y='PL',width=1.0,figsize=(12,8),color=(group.PL>0).map({True:'g',False:'r'}))
    plt.title(f'{tick} [Net PL: {net}]',color='r' if net<0 else 'g',fontsize=15)
    ax.bar_label(ax.containers[0], label_type='edge',color='blue')
    ax1=ax.plot(group['CUMSUM'],marker='o')
    #ax1.bar_label(ax1.containers[1], label_type='edge')
    xval=list(ax1[0].get_data()[0])
    yval=list(ax1[0].get_data()[1])
    yval=[round(i,2) for i in yval]
    for (x,y) in zip(xval,yval):
        c='r' if y<0 else 'g'
        #print(c)
        plt.text(x,y,str(y),color=c,fontfamily='serif',rotation=0)
    plt.tight_layout()
    plt.show()


# STOP HERE

In [ ]:
def get_new_plot(sum_df):
    plt.rcParams['font.size'] = 15
    net=round(sum_df.PL.sum(),2)
    ax=sum_df.plot(kind='bar',x='Name',y='PL',figsize=(16,10),color=(sum_df.PL>0).map({True:'g',False:'r'}))
    plt.title(f'Net PL: {net}',color='r' if net<0 else 'b',fontsize=15)
    ax.bar_label(ax.containers[0], label_type='edge')
    ax1=ax.plot(cum_sum,marker='o')
    plt.tight_layout()
    xval=list(ax1[0].get_data()[0])
    yval=list(ax1[0].get_data()[1])
    yval=[round(i,2) for i in yval]
    for (x,y) in zip(xval,yval):
        c='r' if y<0 else 'g'
        plt.text(x,y,str(y),color=c,fontfamily='serif',rotation=30)
    plt.show()

In [ ]:
sum_df.head()

In [ ]:
df['Name']=df.apply(lambda x:get_label(x['Name'],x['Filled Time']),axis=1)

In [ ]:
df['CallPut']=df['Name'].apply(lambda x:x.split('-')[1])

In [ ]:
df['Placed Time']=pd.to_datetime(df['Placed Time']);df['Filled Time']=pd.to_datetime(df['Filled Time']);
df.head()

In [ ]:
df['Sold Price']=df['Avg Price'].shift(-1);df['Sold Time']=df['Filled Time'].shift(-1)

In [ ]:
df.head()

In [ ]:
# df['Side'].where(df['Side']=='Buy')
df.loc[:,'hold_time']=np.where(df['Side'].isin(['Buy']),(df['Sold Time']-df['Filled Time']).dt.seconds.divide(60).round(2),0)

In [ ]:
df.head()

In [ ]:
df.loc[:,'PL']=np.where(df['Side'].isin(['Buy']),df['Total Qty']*(df['Sold Price']-df['Avg Price']),0)*100

In [ ]:
positive_mean=df.query('PL >0')['PL'].mean().round(2);negative_mean=df.query('PL <0')['PL'].mean().round(2);
positive_mean,negative_mean

In [ ]:
df=df.query('PL!=0.00 or `hold_time`!=0.00')

In [ ]:
#mi=df['PL'].min();ma=df['PL'].max();av=df['PL'].mean()
net=(df['PL'].sum().round(2))-2*df['PL'].count()*0.05
net

In [ ]:
co='r' if net<0 else 'g'


In [ ]:
profit=(df.query('PL>0')['PL'].sum().round(2))
profit

In [ ]:
loss=(df.query('PL<0')['PL'].sum().round(2))
loss

In [ ]:
df['PL'].describe()

In [ ]:
df.loc[:,'PN']=df['PL']>0


In [ ]:
df['PL%']=round(df['PL']/df['Avg Price'],2)
df=df.reset_index(drop=True)
df.head(20)

In [ ]:
df_summary=df[['Name','Side','Total Qty','Avg Price','Sold Price','PL','PL%','Filled Time',
        'Sold Time','hold_time', 'PN','CallPut']].copy()
df_summary.loc[:,'Filled Time']=df['Filled Time'].apply(lambda x:x.strftime('%H:%M:%S'))
df_summary.loc[:,'Sold Time']=df['Sold Time'].apply(lambda x:x.strftime('%H:%M:%S'))
df_summary

In [ ]:
def get_PL_pct(df):
    ax=df.plot(x='Name',y='PL%',kind='bar',width=1.0,figsize=(15,10),rot=90,color=df.PN.map({True:'g',False:'r'}))

    ax.axhline(positive_mean_pct,color='g',linestyle='--',label=positive_mean_pct)
    ax.axhline(negative_mean_pct,color='r',linestyle='--',label=negative_mean_pct)
    for container in ax.containers:
        ax.bar_label(container)

    for item in ax.get_xticklabels():
                item.set_rotation(90)
                item.set_fontsize(14)
                item.set_fontweight('bold')
                item.set_color('blue')

    plt.legend()
    plt.title('PL%')

    plt.show()

In [ ]:
def get_PL(df):
    ax=df.plot(x='Name',y='PL',kind='bar',figsize=(15,10),rot=90,width=1.0,color=df.PN.map({True:'g',False:'r'}))
    #ax.plot(raw_data['Close'],secondary_y=True)
    ax.axhline(positive_mean,color='g',linestyle=':',label=positive_mean)
    ax.axhline(negative_mean,color='r',linestyle=':',label=negative_mean)
    for container in ax.containers:
        ax.bar_label(container)

    for item in ax.get_xticklabels():
                item.set_rotation(90)
                item.set_fontsize(14)
                item.set_fontweight('bold')
                item.set_color('blue')

    plt.legend()
    #plt.text(5,10,'test')
    #plt.text(10,30,f"Profit  no :  {p}\nLoss   no :  {l}",backgroundcolor='k',fontsize=16,color='white')
    plt.text(10,30,printstr,backgroundcolor='k',fontsize=16,color='white')
    plt.text(10,25,"PL: "+str(net),color=co,fontsize=20,backgroundcolor='white')
    plt.title("PL")

    plt.show()

In [ ]:

def get_hold_time(df):
    ax=df.plot(x='Name',y='hold_time',kind='bar',figsize=(12,8),rot=90,width=0.9)#,color=df.PN.map({True:'g',False:'r'}))

    ax.axhline(hold_time_avg,color='r',linestyle='--',label=str(hold_time_avg)+" m")
    #ax.axhline(negative_mean_pct,color='k',linestyle='--',label=negative_mean_pct)
    for container in ax.containers:
        ax.bar_label(container)

    for item in ax.get_xticklabels():
                item.set_rotation(90)
                item.set_fontsize(14)
                item.set_fontweight('bold')
                item.set_color('blue')

    plt.legend()
    plt.title('HOLD TIME')

    plt.show()

In [ ]:
# all plots together
def all_at_once(df):
    fig, axs = plt.subplots(3, 1, figsize=(15,10), sharex=True)

    axs[0].bar(df['Name'],df['PL'],width=1.0,color=df.PN.map({True:'g',False:'r'}))
    axs[0].set_ylabel('PL ($) ')
    axs[0].axhline(positive_mean,color='g',linestyle=':',label=str(positive_mean)+" $")
    axs[0].axhline(negative_mean,color='r',linestyle=':',label=str(negative_mean)+" $")
    #axs[0].text(10,30,f"Profit  no :  {p}\nLoss   no :  {l}",backgroundcolor='k',fontsize=16,color='white')
    #axs[0].text(5,30,printstr,fontsize=16,color='white')
    axs[0].text(10,30,printstr,backgroundcolor='k',fontsize=12,color='white')
    axs[0].text(12,30,"PL: "+str(net),color=co,fontsize=12,backgroundcolor='white')
    axs[0].legend()

    axs[2].bar(df['Name'],df['hold_time'],width=1.0)
    axs[2].set_ylabel('HOLD TIME (mins)')
    axs[2].axhline(hold_time_avg,color='r',linestyle='--',label=str(hold_time_avg)+" m")
    axs[2].legend()

    axs[1].bar(df['Name'],df['PL%'],width=1.0,color=df.PN.map({True:'g',False:'r'}))
    axs[1].axhline(positive_mean_pct,color='g',linestyle='--',label=str(positive_mean_pct)+" %")
    axs[1].axhline(negative_mean_pct,color='r',linestyle='--',label=str(negative_mean_pct)+" %")
    axs[1].set_ylabel('PL (%) ')
    axs[1].legend()

    for ax in axs:
        for container in ax.containers:
            ax.bar_label(container)

    fig.suptitle(f'ALL PLOT {today}')
    plt.xticks(rotation=90,color='blue',fontweight='bold',fontsize=14)
    plt.tight_layout()
    plt.savefig(f'{df}all.pdf')
    plt.show()

In [ ]:
call_put_df=df_summary.CallPut.value_counts().to_frame()

ax=call_put_df.plot(kind='bar')
for container in ax.containers:
        ax.bar_label(container)

In [ ]:
df_call=df_summary[df_summary.CallPut=='Call']
df_call

In [ ]:
df_call.describe()

In [ ]:
get_PL(df_call);get_PL_pct(df_call);get_hold_time(df_call)
#all_at_once(df_call)

In [ ]:
df_put=df_summary[df_summary.CallPut=='Put']
df_put

In [ ]:
df_put.describe()

In [ ]:
get_PL(df_put);get_PL_pct(df_put);get_hold_time(df_put)

In [ ]:
df_summary.round(2).to_csv('test.csv')

In [ ]:
df['PL%'].mean().round(2)

In [ ]:
temp_df=df_summary[['Name','PL','PL%','hold_time','PN']]
temp_df=temp_df.set_index('Name')
temp_df.describe().round(2)


In [ ]:
p=len(df.query('PL>0'));l=len(df.query('PL<0'))
printstr=f"Profit no: {p}\nLoss no  : {l}\nTotal      : {p+l}"

#print(f"Profit no: {p}\nLoss no  : {l}\nTotal    : {p+l}")

print(printstr)

In [ ]:

# ax=df.plot(x='Name',y='PL',kind='bar',figsize=(12,8),rot=90,width=1.0,color=df.PN.map({True:'g',False:'r'}))
# #ax.plot(raw_data['Close'],secondary_y=True)
# ax.axhline(positive_mean,color='g',linestyle=':',label=positive_mean)
# ax.axhline(negative_mean,color='r',linestyle=':',label=negative_mean)
# for container in ax.containers:
#     ax.bar_label(container)

# for item in ax.get_xticklabels():
#             item.set_rotation(90)
#             item.set_fontsize(14)
#             item.set_fontweight('bold')
#             item.set_color('blue')

# plt.legend()
# #plt.text(5,10,'test')
# plt.text(10,30,f"Profit  no :  {p}\nLoss   no :  {l}",backgroundcolor='k',fontsize=16,color='white')
# plt.title("PL")

# plt.show()

In [ ]:
positive_mean_pct=df.query('`PL%` >0')['PL%'].mean().round(2);negative_mean_pct=df.query('`PL%` <0')['PL%'].mean().round(2);hold_time_avg=df['hold_time'].mean().round(2)
positive_mean_pct,negative_mean_pct,hold_time_avg

In [ ]:
get_PL(df);get_PL_pct(df);get_hold_time(df)

In [ ]:

# ax=df.plot(x='Name',y='PL%',kind='bar',width=1.0,figsize=(12,8),rot=90,color=df.PN.map({True:'g',False:'r'}))

# ax.axhline(positive_mean_pct,color='g',linestyle='--',label=positive_mean_pct)
# ax.axhline(negative_mean_pct,color='r',linestyle='--',label=negative_mean_pct)
# for container in ax.containers:
#     ax.bar_label(container)

# for item in ax.get_xticklabels():
#             item.set_rotation(90)
#             item.set_fontsize(14)
#             item.set_fontweight('bold')
#             item.set_color('blue')

# plt.legend()
# plt.title('PL%')

# plt.show()

In [ ]:
# all plots together
fig, axs = plt.subplots(3, 1, figsize=(15,10), sharex=True)

axs[0].bar(df['Name'],df['PL'],width=1.0,color=df.PN.map({True:'g',False:'r'}))
axs[0].set_ylabel('PL ($) ')
axs[0].axhline(positive_mean,color='g',linestyle=':',label=str(positive_mean)+" $")
axs[0].axhline(negative_mean,color='r',linestyle=':',label=str(negative_mean)+" $")
#axs[0].text(10,30,f"Profit  no :  {p}\nLoss   no :  {l}",backgroundcolor='k',fontsize=16,color='white')
#axs[0].text(5,30,printstr,fontsize=16,color='white')
axs[0].text(10,30,printstr,backgroundcolor='k',fontsize=12,color='white')
axs[0].text(12,30,"PL: "+str(net),color=co,fontsize=12,backgroundcolor='white')
axs[0].legend()

axs[2].bar(df['Name'],df['hold_time'],width=1.0)
axs[2].set_ylabel('HOLD TIME (mins)')
axs[2].axhline(hold_time_avg,color='r',linestyle='--',label=str(hold_time_avg)+" m")
axs[2].legend()

axs[1].bar(df['Name'],df['PL%'],width=1.0,color=df.PN.map({True:'g',False:'r'}))
axs[1].axhline(positive_mean_pct,color='g',linestyle='--',label=str(positive_mean_pct)+" %")
axs[1].axhline(negative_mean_pct,color='r',linestyle='--',label=str(negative_mean_pct)+" %")
axs[1].set_ylabel('PL (%) ')
axs[1].legend()

for ax in axs:
    for container in ax.containers:
        ax.bar_label(container)

fig.suptitle(f'ALL PLOT {today}')
plt.xticks(rotation=90,color='blue',fontweight='bold',fontsize=14)
plt.tight_layout()
plt.savefig('all.pdf')
plt.show()

In [ ]:
# # df_sp=pd.read_csv('/home/thakur/test/snp500/SPY.csv')
# # df_sp.head()
# import yfinance as yf

# raw_data = yf.download (tickers = "SPY", period='1d', interval = "1m")

In [ ]:
df_time=df_summary.copy()

In [ ]:
df_time

In [ ]:
df_time['cumsum']=df_time['PL'].cumsum()

In [ ]:
ax=df_time.plot(x='Filled Time',y='cumsum',rot=90,figsize=(20,10))#,xticks=idx)
#ax.set_xticklabels(df_time['Filled Time'])
#plt.xticks(rotation=90)
plt.title(f'Cumsum {today}')
plt.tight_layout()
plt.show()

In [ ]:
kwargs={'color':'r', 'fontsize':14}
kwargs1={'color':'b', 'linestyle':'-','lw':2}
ax=df_time.plot(y='cumsum',kind='line',rot=90,marker='o',figsize=(16,8),**kwargs1).set_xticks(df_time.index,df_time['Filled Time'],**kwargs)#color='r',fontsize=14)#,xticks=idx)
#ax.set_xticklabels(df_time['Filled Time'])
#plt.xticks(rotation=90)

plt.title(f'Cumsum {today}')
plt.ylabel('PL')
plt.xlabel('Time')
plt.tight_layout()
plt.savefig("cumsum.pdf")
plt.show()

In [ ]:
# idx=df_time['Filled Time'].astype('datetime64[ns]').apply(lambda x:x.strftime("%H-%M"))
# idx
# x=[1,2,3]
# y=[3,4,5]
# dict(x,y)

In [ ]:
# idx=pd.to_datetime(df_time['Filled Time']).strftime('%HH:%MM')#.dt.time#.astype('datetime64[ns]')
# idx

In [ ]:
# idx.dtypes

In [ ]:
df_time

In [ ]:
# df_time['cumsum'].plot()

In [ ]:
# df_time['Filled Time']=pd.to_datetime(df['Filled Time'])
# df_time['Sold Time']=pd.to_datetime(df['Sold Time'])
investment=100*df['Avg Price'].sum()
investment,net,net*100/investment


In [ ]:
df['PL%'].mean()

In [ ]:
df_time.dtypes

In [ ]:
#df_time.plot(x='Filled Time',y='PL',rot=90,kind='scatter')
df_time.set_index('Filled Time',inplace=True)
df_time.loc[:,'scumsum']=df_time['cumsum']>0
df_time=df_time[::-1]
#df_time['Avg Price'].plot()


In [ ]:
df_time

In [ ]:
#df_time.loc[:,'scumsum']=df_time['cumsum']>0

In [ ]:
#df_time=df_time[::-1]

In [ ]:
#raw_data.plot(y='Adj Close')

ax1=df_time[::-1][['PL','cumsum']].plot(kind='bar',title=f"Cumsum {today}",grid=True,figsize=(20,10))#,color=df_time['scumsum'].map({True:'g',False:'r'}))
for container in ax1.containers:
        ax1.bar_label(container)
ax1.set_xlabel('PL')
ax1.set_ylabel('Time')
plt.savefig('cumsumplbar.pdf')
plt.show()

In [ ]:
#raw_data.plot(y='Adj Close')
#df_time=df_time[::-1]
ax1=df_time['cumsum'].plot(kind='barh',title=f"Cumsum {today}",grid=True,figsize=(20,10),color=df_time['scumsum'].map({True:'g',False:'r'}))
for container in ax1.containers:
        ax1.bar_label(container)
ax1.set_xlabel('PL')
ax1.set_ylabel('Time')
plt.savefig('cumsumbar.pdf')
plt.show()

In [ ]:
df_time['cumsum'].plot(kind='barh')
#raw_data.head()
# ax.plot(raw_data['Close'])
# plt.show()
# df_time['Avg Price'].plot(color='r')
# df_time['Sold Time'].plot(color='b')

In [ ]:
# #df.plot(x='Name',y='PL',kind='bar')
# x=df['Name'];y=df['PL']
# df.plot(x,y)
#g=sns.barplot(data=df,x=x,y=y)
# for item in g.get_xticklabels():
#     item.set_rotation(90)
#     item.set_fontsize(10)
#     item.set_fontweight('bold')
# for i in g.containers:
#     g.bar_label(i,rotation=90,fontsize=10,fontweight='bold',color='b',padding=5)


In [ ]:
def compare_bars(temp_df,rows=30,title=""):
    choose=['TICKER','CP','VOL','%CHG','%ATR']
    plot_df=temp_df[choose].sort_values(by='CP',ascending=True).reset_index(drop=True,).head(rows)
    columns=plot_df.columns.to_list()
    x=columns[0];y=columns[1:]
    today=datetime.now().strftime("%m/%d/%Y")
    f, axs = plt.subplots(2, 2, figsize=(16, 10), gridspec_kw=dict(width_ratios=[4, 4]))
    cols=axs.flat
    for i,j in enumerate(y):
        g=sns.barplot(data=plot_df,x=x,y=y[i],ax=cols[i])
        for item in g.get_xticklabels():
            item.set_rotation(90)
            item.set_fontsize(10)
            item.set_fontweight('bold')
        for i in g.containers:
            g.bar_label(i,rotation=90,fontsize=10,fontweight='bold',color='b',padding=5)
    f.suptitle(f"{title.upper()} {today}")   
    f.tight_layout()
    #f.savefig(title+'.pdf')
    #f.show()